In [1]:
from __future__ import print_function
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from data import VISDRONEDetection, VISDRONEAnnotationTransform, VISDRONE_ROOT, BaseTransform
from data import VISDRONE_CLASSES as labelmap
import torch.utils.data as data
from models.refinedet import build_refinedet
import warnings
warnings.filterwarnings("ignore")
import sys
import os
import time
import argparse
import numpy as np
import pickle
%matplotlib inline
from matplotlib import pyplot as plt
import time
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
print(torch.cuda.is_available())

True


In [2]:
trained_model = './weights/RefineDet512_VISDRONE_final.pth'
net = build_refinedet('test', 512, 13)    # initialize SSD
net.load_state_dict(torch.load(trained_model))
net.eval()
net = net.cuda()
cudnn.benchmark = True

passed # classes =  13
size =  512
Entered Prior box class


In [3]:
def imTotensor(im, size): # conver an image to a tensor
    x = cv2.resize(im, (size, size)).astype(np.float32)
    x -= (104.0, 117.0, 123.0)
    x = x.astype(np.float32)
    x = x[:, :, ::-1].copy()
    x = torch.from_numpy(x).permute(2, 0, 1).cuda() # cuda()
    print(x.type())
    return x

In [4]:
def StackTens(x): # transform an array of 3d tensors to a single 4d tensor along 0th dimension   
    xx = torch.stack(x, dim=0)
    if torch.cuda.is_available():
        xx = xx.cuda()
    return xx

In [5]:
def TensToNet(net, xx): # pass a 4d tensor to the net
    y = net(xx)
    return y

In [6]:
def putDets(detections, im):
    top_k=10
    colors = plt.cm.hsv(np.linspace(0, 1, 13)).tolist()
    font = cv2.FONT_HERSHEY_SIMPLEX 
#     scale = torch.Tensor(im.shape[1::-1]).repeat(2)
    for k in range(detections.size(0)):
        scale = torch.Tensor(im[k].shape[1::-1]).repeat(2)
        for i in range(detections.size(1)):
            j = 0
            while detections[k,i,j,0] >= 0.80:
                score = detections[k,i,j,0]
                label_name = labelmap[i-1]
                display_txt = '%s: %.2f'%(label_name, score)
                pt = (detections[k,i,j,1:]*scale).cpu().numpy()
                coords = (pt[0], pt[1]), pt[2]-pt[0]+1, pt[3]-pt[1]+1
                color = colors[i]
                cv2.rectangle(im[k], (pt[0], pt[1]), (pt[2], pt[3]), color, 1)
                cv2.putText(im[k], display_txt, (pt[0], pt[1]), font, 1, color, 1)
                j+=1
    return im

In [7]:
def putDets1(detections, im):
    top_k=10
    print(detections.type())
    colors = plt.cm.hsv(np.linspace(0, 1, 13)).tolist()
    font = cv2.FONT_HERSHEY_SIMPLEX 
    scale = torch.Tensor(im.shape[1::-1]).repeat(2)
    for i in range(detections.size(1)):
        j = 0
        while detections[0,i,j,0] >= 0.60:
            score = detections[0,i,j,0]
            label_name = labelmap[i-1]
            display_txt = '%s: %.2f'%(label_name, score)
            pt = (detections[0,i,j,1:]*scale).cpu().numpy()
            coords = (pt[0], pt[1]), pt[2]-pt[0]+1, pt[3]-pt[1]+1
            color = colors[i]
            cv2.rectangle(im, (pt[0], pt[1]), (pt[2], pt[3]), color, 1)
            cv2.putText(im, display_txt, (pt[0], pt[1]), font, 1, color, 1)
            j+=1
    return im

In [8]:
cap = cv2.VideoCapture('drone_360.mp4')
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('Frame count:', frame_count)
cap.set(cv2.CAP_PROP_POS_FRAMES, 1000) # vid.mp4 - 4470 drone_360 - 1000
if (cap.isOpened() == False):
    print('error')
i = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    i += 1
    if ret == True:
        start = time.time()
        x = imTotensor(frame, 512)
        xx = torch.unsqueeze(x, 0)
#         start = time.time()
        y = net(xx)
#         end = time.time()
        det_im = putDets1(y.data, frame)
        cv2.imshow('det', det_im)
        end = time.time()
#         end = time.time()
        print('overall time per frame: ', end-start)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()

Frame count: 2889
torch.cuda.FloatTensor
time spent on NMS:  0.20909547805786133
torch.cuda.FloatTensor
overall time per frame:  2.7205605506896973
torch.cuda.FloatTensor
time spent on NMS:  0.23221969604492188
torch.cuda.FloatTensor
overall time per frame:  0.29610514640808105
torch.cuda.FloatTensor
time spent on NMS:  0.21880578994750977
torch.cuda.FloatTensor
overall time per frame:  0.26031923294067383
torch.cuda.FloatTensor
time spent on NMS:  0.2071218490600586
torch.cuda.FloatTensor
overall time per frame:  0.2483668327331543
torch.cuda.FloatTensor
time spent on NMS:  0.19713401794433594
torch.cuda.FloatTensor
overall time per frame:  0.24207735061645508
torch.cuda.FloatTensor
time spent on NMS:  0.20743536949157715
torch.cuda.FloatTensor
overall time per frame:  0.24811577796936035
torch.cuda.FloatTensor
time spent on NMS:  0.20186257362365723
torch.cuda.FloatTensor
overall time per frame:  0.24309134483337402
torch.cuda.FloatTensor
time spent on NMS:  0.2137002944946289
torch.